###Install Python Packages 

In [1]:
pip install pyproj

###Import Libararies

In [2]:
import pandas as pd
from pyproj import Transformer, Proj
import matplotlib.pyplot as plt
import seaborn as sn
from google.colab import files
import math
import numpy as np

###Load Data Set from Google Drive (shanika.amarasoma@gmail.com)

In [3]:
df = pd.read_excel('https://drive.google.com/uc?id=1UjyHtT5ZB3ci-Mpe3QwO5o5rACtzV2QY', header=0)

In [4]:
df.head()

,OBJECTID,fih_fire_s,fih_year1,fih_season,fih_distri,fih_hist_d,fih_number,fih_fire_t,fih_date1,fih_cause,fih_ignit_,fih_detect,fih_capt_m,fih_author,fih_poly_t,fih_commen,fih_name,fih_burn_p,fih_master,fih_perime,fih_hectar,Shape_Leng,ORIG_FID,POINT_X,POINT_Y
0,76,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,2.750351,21.466456,3250.633742,75,1.305010e+07,-3.779913e+06
1,86,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,1.503180,12.906913,1777.110592,85,1.305081e+07,-3.779902e+06
2,87,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,2.682503,25.813662,3170.978248,86,1.305029e+07,-3.780322e+06
3,88,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,1.593285,14.943114,1884.373850,87,1.305097e+07,-3.780307e+06
4,89,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,2.091454,21.959848,2471.585867,88,1.305049e+07,-3.780735e+06


####Filter data greater than year 2000

In [4]:
#We don't have netcdf files for 2000
df=df[df['fih_year1']>2000]

In [6]:
df.describe()

,OBJECTID,fih_year1,fih_cause,fih_detect,fih_master,fih_perime,fih_hectar,Shape_Leng,ORIG_FID,POINT_X,POINT_Y
count,55961.000000,55961.000000,55961.000000,55961.000000,5.596100e+04,5.596100e+04,5.596100e+04,5.596100e+04,55961.000000,5.596100e+04,5.596100e+04
mean,105944.503172,2009.772413,443.870195,22.920963,1.333446e+06,1.414151e+03,1.927132e+03,4.783722e+04,105943.503172,1.330868e+07,-3.284709e+06
std,22222.892706,5.573432,494.817580,142.268314,3.399475e+06,6.020197e+04,2.673340e+04,4.240275e+05,22222.892706,4.208617e+05,6.225067e+05
min,76.000000,2001.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,2.700000e-07,3.128805e-01,75.000000,1.260127e+07,-4.180769e+06
25%,90759.000000,2005.000000,0.000000,0.000000,0.000000e+00,6.134547e-01,1.711369e+00,7.449937e+02,90758.000000,1.292400e+07,-3.885431e+06
50%,106003.000000,2009.000000,8.000000,0.000000,0.000000e+00,1.829531e+00,8.808259e+00,2.097188e+03,106002.000000,1.316817e+07,-3.364985e+06
75%,122503.000000,2015.000000,999.000000,0.000000,0.000000e+00,9.520545e+00,8.241568e+01,8.961516e+03,122502.000000,1.364321e+07,-2.636406e+06
max,141759.000000,2020.000000,999.000000,5000.000000,9.999999e+06,1.053722e+07,2.769614e+06,4.054606e+07,141758.000000,1.442805e+07,-1.591364e+06


###Convert X, Y to Lat, Lon and Hectare to Acre

In [7]:
transformer = Transformer.from_crs("epsg:3857","epsg:4326")
df['LAT'],df['LON']=transformer.transform(df['POINT_X'],df['POINT_Y'])
df['ACRES']=df['fih_hectar']*2.47105
df['LON']=df['LON'].round(4)
df['LAT']=df['LAT'].round(4)
df['ACRES']=df['ACRES'].round(4)

### MODIS Tile Calculator

In [8]:
#https://gis.stackexchange.com/questions/265400/getting-tile-number-of-sinusoidal-modis-product-from-lat-long
CELLS = 2400
VERTICAL_TILES = 18
HORIZONTAL_TILES = 36
EARTH_RADIUS = 6371007.181
EARTH_WIDTH = 2 * math.pi * EARTH_RADIUS
TILE_WIDTH = EARTH_WIDTH / HORIZONTAL_TILES
TILE_HEIGHT = TILE_WIDTH
CELL_SIZE = TILE_WIDTH / CELLS
MODIS_GRID = Proj(f'+proj=sinu +R={EARTH_RADIUS} +nadgrids=@null +wktext')
def lat_lon_to_modis(lat, lon):
    x, y = MODIS_GRID(lon, lat)
    h = (EARTH_WIDTH * .5 + x) / TILE_WIDTH
    v = -(EARTH_WIDTH * .25 + y - (VERTICAL_TILES - 0) * TILE_HEIGHT) / TILE_HEIGHT
    return 'h'+str(int(h))+'v'+str(int(v))

In [9]:
df['MODIS']=df.apply(lambda x: lat_lon_to_modis(x.LAT, x.LON), axis=1)

In [10]:
df.head()

,OBJECTID,fih_fire_s,fih_year1,fih_season,fih_distri,fih_hist_d,fih_number,fih_fire_t,fih_date1,fih_cause,fih_ignit_,fih_detect,fih_capt_m,fih_author,fih_poly_t,fih_commen,fih_name,fih_burn_p,fih_master,fih_perime,fih_hectar,Shape_Leng,ORIG_FID,POINT_X,POINT_Y,LAT,LON,ACRES,MODIS
0,76,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,2.750351,21.466456,3250.633742,75,1.305010e+07,-3.779913e+06,-32.1264,117.2310,53.0447,h27v12
1,86,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,1.503180,12.906913,1777.110592,85,1.305081e+07,-3.779902e+06,-32.1263,117.2375,31.8936,h27v12
2,87,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,2.682503,25.813662,3170.978248,86,1.305029e+07,-3.780322e+06,-32.1295,117.2328,63.7869,h27v12
3,88,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,1.593285,14.943114,1884.373850,87,1.305097e+07,-3.780307e+06,-32.1294,117.2389,36.9252,h27v12
4,89,2008/2009,2008,ND,CWB,,999,PL,2008-07-01,0,,0,,vickyr,,FPC Plantations data,,,0,2.091454,21.959848,2471.585867,88,1.305049e+07,-3.780735e+06,-32.1326,117.2345,54.2639,h27v12


In [11]:
df.groupby(['MODIS']).size()

MODIS
h27v11       78
h27v12    22840
h28v11     9104
h28v12     3040
h29v10      171
h29v11    20206
h29v12        5
h30v10      517
dtype: int64

#Filter out Less values tiles

In [12]:
df=df[(df['MODIS']=='h27v12') | (df['MODIS']=='h28v11') | (df['MODIS']=='h29v11')]

In [13]:
df.shape

(52150, 29)

In [14]:
df.groupby(['fih_year1','MODIS']).size()

fih_year1  MODIS 
2001       h27v12    1202
           h28v11     971
           h29v11     818
2002       h27v12    1485
           h28v11     586
           h29v11    1468
2003       h27v12    1242
           h28v11     371
           h29v11    1800
2004       h27v12    1680
           h28v11     309
           h29v11    1287
2005       h27v12    1192
           h28v11     150
           h29v11     225
2006       h27v12    1708
           h28v11     318
           h29v11     976
2007       h27v12    1473
           h28v11     214
           h29v11     719
2008       h27v12    1562
           h28v11     260
           h29v11     303
2009       h27v12    3420
           h28v11     207
           h29v11     726
2010       h27v12     866
           h28v11     114
           h29v11     192
2011       h27v12     925
           h28v11     187
           h29v11     219
2012       h27v12     966
           h28v11    1397
           h29v11    2453
2013       h27v12     763
           h28v11   

### Drop Unwanted Colums

In [15]:
df=df.drop(['OBJECTID', 'fih_burn_p', 'fih_master', 'fih_hectar', 'Shape_Leng', 'ORIG_FID', 'POINT_X', 'POINT_Y'], axis=1)

In [16]:
df.to_excel('WesternAustraliaFireData-ver2.xlsx',index=False)

In [17]:
files.download("/content/WesternAustraliaFireData-ver2.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Unique Date, Lat, Lon, Modis Tile

In [18]:
df1=df[['fih_year1', 'fih_date1', 'LAT', 'LON', 'MODIS']].drop_duplicates().sort_values(by=['fih_date1', 'MODIS'])

In [19]:
df1.to_excel('WesternAustraliaFireDataUniqDateLatLonModis.xlsx',index=False)

In [20]:
files.download("/content/WesternAustraliaFireDataUniqDateLatLonModis.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Unique Lat, Lon

In [21]:
df2=df[['LAT', 'LON', 'MODIS']].drop_duplicates()

In [22]:
df2.to_excel('WesternAustraliaFireDataUniqLatLonModis.xlsx',index=False)

In [23]:
files.download("/content/WesternAustraliaFireDataUniqLatLonModis.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>